In [2]:
import os
import zipfile
import json

import pandas as pd
import numpy as np

In [10]:
EXPORT_ZIP_PATH = 'export.zip'
TARGET_DIRECTORY = './data'

with zipfile.ZipFile(EXPORT_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(TARGET_DIRECTORY)

In [12]:
EXPORT_XML = TARGET_DIRECTORY + '/apple_health_export/export.xml'

if not os.path.isfile(EXPORT_XML):
    raise 'Could not find xml export'

df = pd.read_xml(EXPORT_XML, xpath='//Record[@type="HKQuantityTypeIdentifierHeartRate"]')
df = df[['startDate', 'value']]

In [13]:
export = df.rename(columns={'startDate': 'x', 'value': 'y'})
export['x'] = pd.to_datetime(export['x'])
export = export.set_index('x')
export = export.resample('H').median()
export = export.reset_index()
export['x'] = export['x'].astype(np.int64) // 10 ** 6

/tmp/ipykernel_45608/1150266851.py:6: FutureWarning: casting datetime64[ns, pytz.FixedOffset(120)] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  export['x'] = export['x'].astype(np.int64) // 10 ** 6


In [14]:
with open('./data.json', 'w') as export_json:
    export.to_json(path_or_buf=export_json, orient='records')

with open('./stats.json', 'w') as export_json:
    json.dump({
        "quantile_95": round(export['y'].quantile(.95)),
        "quantile_99": round(export['y'].quantile(.01)),
        "quantile_range": {
            "end": round(export['y'].quantile(.99)),
            "start": round(export['y'].quantile(.01))
        },
        "mean": round(export['y'].mean()),
        "median": round(export['y'].median())
    }, export_json)

In [26]:
with open('./hist.json', 'w') as export_json:
    bins = {}
    hist = {}
    
    percentile_99 = export['y'].quantile(.99)
    normalized_export = export[export['y'] < percentile_99]['y']
    
    bins['default'] = np.linspace(normalized_export.min(), normalized_export.max(), 10)
    hist['default'] = np.histogram(np.array(normalized_export), bins['default'])
    
    export_sqrt = np.sqrt((normalized_export))
    bins['sqrt'] = np.linspace(export_sqrt.min(), export_sqrt.max(), 10)
    hist['sqrt'] = np.histogram(np.array(export_sqrt), bins['sqrt'])
    
    export_log = np.log((normalized_export))
    bins['log'] = np.linspace(export_log.min(), export_log.max(), 10)
    hist['log'] = np.histogram(np.array(export_log), bins['log'])

    json.dump({
        "default": {
            "count": hist['default'][0].tolist(),
            "bucket": [round(num, 2) for num in hist['default'][1].tolist()]
        },
        "sqrt": {
            "count": hist['sqrt'][0].tolist(),
            "bucket": [round(num, 2) for num in hist['sqrt'][1].tolist()]
        },
        "log": {
            "count": hist['log'][0].tolist(),
            "bucket": [round(num, 2) for num in hist['log'][1].tolist()]
        },
    }, export_json)